In [1]:
!python -m venv --copies awalk55

In [2]:
%cd /mnt/beegfs/labs/bozkurtlab/drew-projects/loneliness-ehr/
%ls

/mnt/beegfs/labs/bozkurtlab/drew-projects/loneliness-ehr
acute_social_isolation_llm_labeled.csv
acute_social_isolation_results.csv
ajudication-reliability-files/
annotation_Drew_sample_social_disconnection_DW.csv
annotation_Selen_sample_social_disconnection_SB.csv
awalk55/
chronic_social_isolation_llm_3_shot_labeled.csv
chronic_social_isolation_llm_labeled.csv
chronic_social_isolation_llm_one_shot_labeled.csv
chronic_social_isolation_results.csv
gold_standard_social_isolation_1000.csv
lives_alone_results.csv
lives_alone_social_isolation_llm_labeled.csv
lives_alone_social_isolation_results.csv
summary_models_acute_social_isolation.csv
summary_models_chronic_social_isolation.csv
summary_models_lives_alone.csv
summary_models_lives_alone_social_isolation.csv


/opt/modules/Python/3.11.5/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import requests
import pandas as pd
import numpy as np
%ls

acute_social_isolation_llm_labeled.csv
acute_social_isolation_results.csv
ajudication-reliability-files/
annotation_Drew_sample_social_disconnection_DW.csv
annotation_Selen_sample_social_disconnection_SB.csv
awalk55/
chronic_social_isolation_llm_3_shot_labeled.csv
chronic_social_isolation_llm_labeled.csv
chronic_social_isolation_llm_one_shot_labeled.csv
chronic_social_isolation_results.csv
gold_standard_social_isolation_1000.csv
lives_alone_results.csv
lives_alone_social_isolation_llm_labeled.csv
lives_alone_social_isolation_results.csv
summary_models_acute_social_isolation.csv
summary_models_chronic_social_isolation.csv
summary_models_lives_alone.csv
summary_models_lives_alone_social_isolation.csv


In [6]:

# lives_alone_label, acute_social_disconnection_label
df_concat = pd.read_csv("gold_standard_social_isolation_1000.csv")
df_concat.info()
# Filter out training
exclude_ids = [4265489,1056798] 
exclude_ids = np.array(exclude_ids, dtype=np.int64)
# Exclude the specified Sentence_ID 
df_concat_filtered = df_concat[~df_concat['Sentence ID'].isin(exclude_ids)] 



# Chronic social isolation classifier
chronic_df = df_concat_filtered[['full_text_truncated', 'lives_alone_label']]

# Display the first few rows of chronic_df
chronic_df.info()
chronic_df.head()
unique_chronic_values = chronic_df['lives_alone_label'].value_counts().reset_index()

# Renaming columns for clarity
unique_chronic_values.columns = ['lives_alone_label', 'count']
unique_chronic_values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          1000 non-null   int64  
 1   Sentence ID                         1000 non-null   int64  
 2   TEXT                                1000 non-null   object 
 3   Sentence                            1000 non-null   object 
 4   matched_term                        1000 non-null   object 
 5   chronic_social_disconnection_label  1000 non-null   float64
 6   lives_alone_label                   1000 non-null   float64
 7   acute_social_disconnection_label    1000 non-null   float64
 8   full_text                           1000 non-null   object 
 9   full_text_truncated                 1000 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 78.3+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 998 en

,lives_alone_label,count
0,0.0,811
1,1.0,187


In [7]:
%pip install seaborn
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
train_size = 0.8
train_data=chronic_df.sample(frac=train_size,random_state=0)
test_data=chronic_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(df_concat.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

FULL Dataset: (1000, 10)
TRAIN Dataset: (798, 2)
TEST Dataset: (200, 2)


In [9]:
import requests
import pandas as pd
from datetime import datetime
import time

start_time = time.time()

def request(query):
    # Define the headers
    headers = {
        'Content-Type': 'application/json'
    }
    
    # Define the data
    data = {
        "model": model_name,
        "prompt": query,
        "max_tokens": max_tokens,
        "temperature": 1,
        "top_p": 0.95,
    }
    
    # Make the POST request
    response = requests.post(url, headers=headers, json=data)
    
    # Check if the response is successful
    if response.status_code == 200:
        response_json = response.json()
        
        # Check if the 'choices' key exists
        if 'choices' in response_json:
            answer = response_json['choices'][0]['text']
            print('Answer:\n', answer)
            return answer
        else:
            print("Error: 'choices' key not found in the response.")
            print("Full response content:", response_json)
            return None
    else:
        print(f"Error: Received status code {response.status_code}")
        print("Response content:", response.content)
        return None

def process_dataframe(df, model_name, max_tokens, url):
    results = []
    
    for index, row in df.iterrows():
        prompt = f'''
        You are an expert in abstracting social health from medical language, focused on studying the documentation of social isolation and loneliness within an intensive care unit's electronic health record.
        Read the following sentence, appended by </s> and the full patient note text, and respond with a 1 or 0. Respond with “1” if the chart describes or alludes to a patient living alone. 
        Respond with “0” if the chart does not, if there is not enough evidence of a patient living alone in the chart, or if you are otherwise unsure. 
        After the response, write a short sentence describing the rationale behind the 1/0 classification.

        To assist you with classification, below are one example of a 1-labeled sentence, without the full chart, where patient living alone is present and a 0-labeled sentence where it is not:
        1: Social History: Not married, lives alone.
        0: While the sharp activity may come from the encephalopathy (plus a focal lesion) alone, the rhythmicity and particularly sharp features indicate a risk for seizures, particularly with focal onset in the right frontal region.

        
        Read the following sentence matching a related keyword, which is appended by </s> and followed by the entire chart carefully:
        {row['full_text_truncated']}
        Please strictly format each response beginning with "Response:" followed by either 0 or 1, then "Rationale:" followed by your explanation.'''
        
        answer = request(prompt)
        if answer is not None:
            results.append({'full_text_truncated': row['full_text_truncated'], 'label_model': answer, 'gold_label': row['lives_alone_label']})
        else:
            results.append({'full_text_truncated': row['full_text_truncated'], 'label_model': 'Error', 'gold_label': row['lives_alone_label']})
    
    return pd.DataFrame(results)

# Model parameters
model_name = 'meta-llama/Meta-Llama-3-70B-Instruct'
max_tokens = 100
url = 'http://llm1.priv.bmi.emory.edu:8000/llama3_70B/v1/completions'


# Process the DataFrame
result_df = process_dataframe(test_data, model_name, max_tokens, url)

print(result_df)
end_time = time.time()
duration = end_time - start_time
print(f"{duration} seconds")


Answer:
  Be cautious not to make assumptions about patient living situation. Only use the chart information to determine your classification. 
        Ready when you are!
Error: Received status code 400
Response content: b'{"object":"error","message":"This model\'s maximum context length is 8192 tokens. However, you requested 9204 tokens (9104 in the messages, 100 in the completion). Please reduce the length of the messages or completion.","type":"BadRequestError","param":null,"code":400}'
Answer:
 
Answer:
  In this case, the sentence mentions "Social: Found down in apartment alone," indicating the patient lives alone, hence the 1 classification.
Answer:
  

Response: 1
Rationale: The social history section of the chart mentions that the patient "lived alone at home - retired", which indicates that the patient resides alone.
Answer:
  Do not provide unnecessary text.        
        Response: 1
        Rationale: The patient states he lives alone.
Answer:
 
Answer:
 
Answer:
  

Resp

In [10]:
# Parse responses
import re

# Function to extract the first number (0 or 1) after 'Response:'
def extract_response(text):
    start = text.find("Response:") + len("Response:")
    if start != -1:
        # Extract the text following "Response:"
        text_after_response = text[start:].strip()
        # Use regex to find the first occurrence of 0 or 1
        match = re.search(r'\b[01]\b', text_after_response)
        if match:
            return int(match.group())  # Return the number found (0 or 1)
    return 0  # Default to 0 if no valid response is found

# Function to extract the rationale (all text following 'Rationale:')
def extract_rationale(text):
    start = text.find("Rationale:") + len("Rationale:")
    if start != -1:
        return text[start:].strip()
    return ""  # Return empty string if "Rationale:" is not found

# Apply the functions to create new columns
result_df['response'] = result_df['label_model'].apply(extract_response)
result_df['rationale'] = result_df['label_model'].apply(extract_rationale)



In [11]:
result_df
result_df['response'].value_counts()
result_df['gold_label'].value_counts()

gold_label
0.0    154
1.0     46
Name: count, dtype: int64

In [12]:
result_df.to_csv("lives_alone_social_isolation_llm_one_shot_labeled.csv")

In [13]:
result_df = pd.read_csv("lives_alone_social_isolation_llm_one_shot_labeled.csv")

In [14]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

classificaiton_report = classification_report(result_df['gold_label'],result_df['response'])

print(classificaiton_report)

              precision    recall  f1-score   support

         0.0       0.84      0.90      0.87       154
         1.0       0.57      0.43      0.49        46

    accuracy                           0.80       200
   macro avg       0.71      0.67      0.68       200
weighted avg       0.78      0.80      0.78       200



In [15]:
results = []

for _ in range(1000):
    sample_df = result_df.sample(len(result_df), replace=True)

    preds = sample_df['response'].values
    true_labels = sample_df['gold_label'].values

    accuracy = accuracy_score(true_labels, preds)
    positive_precision = precision_score(true_labels, preds, labels=[1], average='binary')
    positive_recall = recall_score(true_labels, preds, labels=[1], average='binary')
    positive_f1 = f1_score(true_labels, preds, labels=[1], average='binary')

    results.append({
        'Accuracy': accuracy,
        'Positive Precision': positive_precision,
        'Positive Recall': positive_recall,
        'Positive F1': positive_f1,
    })

results_df = pd.DataFrame(results)

# Calculate mean
mean_summary = results_df.mean()

# Calculate confidence intervals
ci_lower = results_df.apply(lambda x: np.percentile(x, 2.5))
ci_upper = results_df.apply(lambda x: np.percentile(x, 97.5))

# Combine mean and confidence intervals into a single DataFrame
summary = pd.DataFrame({
    'Mean': mean_summary,
    '2.5%': ci_lower,
    '97.5%': ci_upper
})

print(summary)



                        Mean      2.5%     97.5%
Accuracy            0.795765  0.740000  0.850000
Positive Precision  0.573047  0.407378  0.750000
Positive Recall     0.436536  0.289448  0.577778
Positive F1         0.492102  0.350877  0.623690
